# СПБ

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline

payed_spb = pd.DataFrame()

def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

for path, folders, files in os.walk(r'C:\Users\AmanovRA\Текущее\Платные каналы\Данные для отчета 1 кв\Спб'):
    if len(files) > 0:
        for file in files:
            source = f'Источник: {os.path.basename(path)}, Разделение: {file[:file.find(".")]}'
            temp_df = pd.read_excel(path + '\\' + file)
            temp_df['Источник'] = [source for i in range(temp_df.shape[0])]
            temp_df = temp_df.loc[:, ['Телефон', 'Источник']]
            temp_df['Телефон'] = temp_df['Телефон'].apply(to_string)
            payed_spb = payed_spb.append(temp_df, ignore_index=True)
            

  
        

phones = payed_spb['Телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 11:
        phones[i] = ''.join([x for x in j if x.isdigit()])

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()
for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) <= 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y
            
payed_spb['Телефон'] = phones2    

def join_list(lst):
    return ', '.join([str(i) for i in lst])

payed_spb['Телефон'] = payed_spb['Телефон'].apply(join_list)

payed_spb = payed_spb.drop_duplicates(subset='Телефон')

payed_spb = payed_spb.reset_index()
payed_spb.drop('index', axis=1, inplace=True)
payed_spb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

au_telefons = pd.read_excel(r'C:\Users\AmanovRA\Текущее\Платные каналы\Данные для отчета 1 кв\АУ_по_июнь.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)                 

lst_with_coincidenc_telefon = []
lst_with_coincidenc_source = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append([])
    lst_with_coincidenc_source.append([])
    for i, j in enumerate(payed_spb['Телефон']):
        if str(j) in row:
            lst_with_coincidenc_telefon[index].append(str(j))
            lst_with_coincidenc_source[index].append(payed_spb['Источник'][i])
            
au_telefons['Payed-телефон'], au_telefons['Источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source         
            
def join_list(lst):
    return ', '.join([str(i) for i in lst])

au_telefons['Payed-телефон'], au_telefons['Источник'] = au_telefons['Payed-телефон'].apply(join_list), au_telefons['Источник'].apply(join_list)


In [62]:
au_telefons.to_excel(r'C:\Users\AmanovRA\Текущее\Платные каналы\Данные для отчета 1 кв\Payed_channels_spb.xlsx')

# Новосибирск

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline

payed_nvs = pd.DataFrame()

def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

for path, folders, files in os.walk(r'C:\Users\AmanovRA\Текущее\Платные каналы\Данные для отчета 1 кв\Нск'):
    if len(files) > 0:
        for file in files:
            source = f'Источник: {os.path.basename(path)}, Разделение: {file[:file.find(".")]}'
            temp_df = pd.read_excel(path + '\\' + file)
            temp_df['Источник'] = [source for i in range(temp_df.shape[0])]
            temp_df = temp_df.loc[:, ['Телефон', 'Источник']]
            temp_df = temp_df.dropna(subset=['Телефон'])
            temp_df['Телефон'] = temp_df['Телефон'].apply(to_string)
            payed_nvs = payed_nvs.append(temp_df, ignore_index=True)
            

  
        

phones = payed_nvs['Телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 11:
        phones[i] = ''.join([x for x in j if x.isdigit()])

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()
for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) <= 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y
            
payed_nvs['Телефон'] = phones2    

def join_list(lst):
    return ', '.join([str(i) for i in lst])

payed_nvs['Телефон'] = payed_nvs['Телефон'].apply(join_list)

payed_nvs = payed_nvs.drop_duplicates(subset='Телефон')

payed_nvs = payed_nvs.reset_index()
payed_nvs.drop('index', axis=1, inplace=True)
payed_nvs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

au_telefons = pd.read_excel(r'C:\Users\AmanovRA\Текущее\Платные каналы\Данные для отчета 1 кв\НСК_по_июнь.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)                 

lst_with_coincidenc_telefon = []
lst_with_coincidenc_source = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append([])
    lst_with_coincidenc_source.append([])
    for i, j in enumerate(payed_nvs['Телефон']):
        if str(j) in row:
            lst_with_coincidenc_telefon[index].append(str(j))
            lst_with_coincidenc_source[index].append(payed_nvs['Источник'][i])
            
au_telefons['Payed-телефон'], au_telefons['Источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source         
            
def join_list(lst):
    return ', '.join([str(i) for i in lst])

au_telefons['Payed-телефон'], au_telefons['Источник'] = au_telefons['Payed-телефон'].apply(join_list), au_telefons['Источник'].apply(join_list)


In [73]:
au_telefons.to_excel(r'C:\Users\AmanovRA\Текущее\Платные каналы\Данные для отчета 1 кв\Payed_channels_nvs.xlsx')